# 项目：评估和清理英国电商公司销售数据

## 分析目标

此数据分析的目的是，根据市场销售数据，挖掘畅销产品，以便制定更有效的市场策略来提升营收。

本实战项目的目的在于练习评估数据干净和整洁度，并且基于评估结果，对数据进行清洗，从而得到可供下一步分析的数据。

## 简介

原始数据集记录了一家英国在线零售公司在2010年12月1日至2011年12月9日期间的所有交易情况，涵盖了该公司在全球不同国家和地区的业务数据。该公司主要销售覆盖各个场景的礼品，包括但不限于生日礼品、结婚纪念品、圣诞礼品等等。该公司的客户群体主要包括批发商和个人消费者，其中批发商占据了相当大的比例。

数据每列的含义如下：
- `InvoiceNo`: 发票号码。6位数，作为交易的唯一标识符。如果这个代码以字母“c”开头，表示这笔交易被取消。
- `StockCode`: 产品代码。5位数，作为产品的唯一标识符。
- `Description`: 产品名称。
- `Quantity`: 产品在交易中的数量。
- `InvoiceDate`: 发票日期和时间。交易发生的日期和时间。
- `UnitPrice`: 单价。价格单位为英镑（£）。
- `CustomerID`: 客户编号。5位数，作为客户的唯一标识符。
- `Country`: 国家名称。客户所居住的国家的名称。

## 1. 读取数据

In [1]:
import pandas as pd

In [2]:
original_data = pd.read_csv("e_commerce.csv")
original_data.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom


## 2. 评估数据

对上面建立的`original_data`这个DataFrame包含的数据进行评估。 

评估两个方面：
1. 结构。数据是否整齐，即“每列一个变量，每行一个观察值，每个单元格是一个值”。
2. 内容。数据是否是脏数据，即“丢失数据、重复数据、不一致数据、无效或错误数据”。

### 2.1 评估数据结构

In [3]:
original_data.sample(10)

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
155164,549974,37495,FAIRY CAKE BIRTHDAY CANDLE SET,4,4/13/2011 14:31,3.75,17746.0,United Kingdom
491721,578072,21810,CHRISTMAS HANGING STAR WITH BELL,96,11/22/2011 16:02,0.39,17759.0,United Kingdom
122692,546884,85170D,SET/6 PINK BIRD T-LIGHT CANDLES,1,3/17/2011 18:08,4.13,NaN,United Kingdom
488389,577816,22087,PAPER BUNTING WHITE LACE,40,11/22/2011 9:17,2.55,18139.0,United Kingdom
100991,544888,84030E,ENGLISH ROSE HOT WATER BOTTLE,4,2/24/2011 11:55,4.25,14231.0,United Kingdom
49070,540500,22423,REGENCY CAKESTAND 3 TIER,16,1/9/2011 10:26,10.95,15465.0,United Kingdom
133528,547788,22183,CAKE STAND VICTORIAN FILIGREE MED,4,3/25/2011 12:00,5.79,NaN,United Kingdom
151562,549559,21246,RED RETROSPOT BIG BOWL,6,4/10/2011 13:25,4.95,12447.0,Belgium
489914,578031,22470,HEART OF WICKER LARGE,6,11/22/2011 13:41,2.95,15796.0,United Kingdom
439954,574491,22623,BOX OF VINTAGE JIGSAW BLOCKS,3,11/4/2011 13:05,5.95,16318.0,United Kingdom


结论：结构没有问题，符合“每列一个变量，每行一个观察值，每个单元格是一个值”。

### 2.2 评估数据内容

In [4]:
original_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   InvoiceNo    541909 non-null  object 
 1   StockCode    541909 non-null  object 
 2   Description  540455 non-null  object 
 3   Quantity     541909 non-null  int64  
 4   InvoiceDate  541909 non-null  object 
 5   UnitPrice    541909 non-null  float64
 6   CustomerID   406829 non-null  float64
 7   Country      541909 non-null  object 
dtypes: float64(2), int64(1), object(5)
memory usage: 33.1+ MB


1. 541909行数据，`Description`与`CustomerID`非空值不等于541909，存在NaN。
2. `InvoiceDate `Dtype=`object`，实际上发票日期应为`datetime`; ` CustomerID `Dtype = `float`，实际上应为`string(object)`。

#### 2.2.1 评估缺失数据

找出`Description`缺失的观察值

In [5]:
original_data[original_data["Description"].isnull()]  

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
622,536414,22139,NaN,56,12/1/2010 11:52,0.0,NaN,United Kingdom
1970,536545,21134,NaN,1,12/1/2010 14:32,0.0,NaN,United Kingdom
1971,536546,22145,NaN,1,12/1/2010 14:33,0.0,NaN,United Kingdom
1972,536547,37509,NaN,1,12/1/2010 14:33,0.0,NaN,United Kingdom
1987,536549,85226A,NaN,1,12/1/2010 14:34,0.0,NaN,United Kingdom
...,...,...,...,...,...,...,...,...
535322,581199,84581,NaN,-2,12/7/2011 18:26,0.0,NaN,United Kingdom
535326,581203,23406,NaN,15,12/7/2011 18:31,0.0,NaN,United Kingdom
535332,581209,21620,NaN,6,12/7/2011 18:35,0.0,NaN,United Kingdom
536981,581234,72817,NaN,27,12/8/2011 10:33,0.0,NaN,United Kingdom


从结果上来看，`Description`空的地方`UnitPrice`都为0，现在我们想验证这一猜想，增加筛选条件，看是否存在`Description`空且`UnitPrice`不为0

In [6]:
original_data[(original_data["Description"].isnull()) & original_data["UnitPrice"]!=0] 

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country


输出为空，猜想正确。缺失`Description`的同时，`UnitPrice`也无效。我们的目标是：根据市场销售数据，挖掘畅销产品，以便制定更有效的市场策略来提升营收，因此缺失`Description`与`UnitPrice`的观察值不会对我们后续的分析产生作用，因此删去。

找出`CustomerID`缺失的观察值

In [7]:
original_data[original_data["CustomerID"].isnull()]

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
622,536414,22139,NaN,56,12/1/2010 11:52,0.00,NaN,United Kingdom
1443,536544,21773,DECORATIVE ROSE BATHROOM BOTTLE,1,12/1/2010 14:32,2.51,NaN,United Kingdom
1444,536544,21774,DECORATIVE CATS BATHROOM BOTTLE,2,12/1/2010 14:32,2.51,NaN,United Kingdom
1445,536544,21786,POLKADOT RAIN HAT,4,12/1/2010 14:32,0.85,NaN,United Kingdom
1446,536544,21787,RAIN PONCHO RETROSPOT,2,12/1/2010 14:32,1.66,NaN,United Kingdom
...,...,...,...,...,...,...,...,...
541536,581498,85099B,JUMBO BAG RED RETROSPOT,5,12/9/2011 10:26,4.13,NaN,United Kingdom
541537,581498,85099C,JUMBO BAG BAROQUE BLACK WHITE,4,12/9/2011 10:26,4.13,NaN,United Kingdom
541538,581498,85150,LADIES & GENTLEMEN METAL SIGN,1,12/9/2011 10:26,4.96,NaN,United Kingdom
541539,581498,85174,S/4 CACTI CANDLES,1,12/9/2011 10:26,10.79,NaN,United Kingdom


`CustomerID`缺失不影响研究畅销商品，并且有些`CustomerID`缺失的观察值仍然是有效的，因此保留变量为空的观察值。

#### 2.2.2 评估重复数据

显然，虽然`InvoiceNo` `StockCode` `CustomerID`为唯一标识符，但是他们在实际场景中均可以重复，我们只需查看他们三个一起是否同时在`original_data`里面重复

In [8]:
original_data[original_data.duplicated(subset=['InvoiceNo','StockCode','CustomerID'],keep = False)]

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
113,536381,71270,PHOTO CLIP LINE,1,12/1/2010 9:41,1.25,15311.0,United Kingdom
125,536381,71270,PHOTO CLIP LINE,3,12/1/2010 9:41,1.25,15311.0,United Kingdom
483,536409,90199C,5 STRAND GLASS NECKLACE CRYSTAL,3,12/1/2010 11:45,6.35,17908.0,United Kingdom
485,536409,22111,SCOTTIE DOG HOT WATER BOTTLE,1,12/1/2010 11:45,4.95,17908.0,United Kingdom
489,536409,22866,HAND WARMER SCOTTY DOG DESIGN,1,12/1/2010 11:45,2.10,17908.0,United Kingdom
...,...,...,...,...,...,...,...,...
541692,581538,22992,REVOLVER WOODEN RULER,1,12/9/2011 11:34,1.95,14446.0,United Kingdom
541697,581538,21194,PINK HONEYCOMB PAPER FAN,1,12/9/2011 11:34,0.65,14446.0,United Kingdom
541698,581538,35004B,SET OF 3 BLACK FLYING DUCKS,1,12/9/2011 11:34,5.45,14446.0,United Kingdom
541699,581538,22694,WICKER STAR,1,12/9/2011 11:34,2.10,14446.0,United Kingdom


我们发现`InvoiceNo` `StockCode` `CustomerID`可以一起重复,因为一次可以分批次购买同一商品,如ID为`536381`的发票上同时购买了几次ID为`71270`的物品，只是数量不同。因此我们不需要再删除重复数据了。

#### 2.2.3 评估不一致数据

只有`Country`才有可能不一致

In [9]:
original_data["Country"].value_counts()

United Kingdom          495266
Germany                   9495
France                    8557
EIRE                      8196
Spain                     2533
Netherlands               2371
Belgium                   2069
Switzerland               2002
Portugal                  1519
Australia                 1259
Norway                    1086
Italy                      803
Channel Islands            758
Finland                    695
Cyprus                     622
Sweden                     462
Unspecified                446
Austria                    401
Denmark                    389
Japan                      358
Poland                     341
Israel                     297
China                      288
Singapore                  229
USA                        218
UK                         211
Iceland                    182
Canada                     151
Greece                     146
Malta                      127
United States               73
United Arab Emirates        68
European

1. `United Kingdom ` `UK` `U.K.`-> 英国。
2. `USA` `United States `->美国。   
只保留一个进行统一。

#### 2.2.4 评估错误数据或无效数据

In [10]:
original_data.describe()

,Quantity,UnitPrice,CustomerID
count,541909.000000,541909.000000,406829.000000
mean,9.552250,4.611114,15287.690570
std,218.081158,96.759853,1713.600303
min,-80995.000000,-11062.060000,12346.000000
25%,1.000000,1.250000,13953.000000
50%,3.000000,2.080000,15152.000000
75%,10.000000,4.130000,16791.000000
max,80995.000000,38970.000000,18287.000000


`Quantity`与`UnitPrice`居然出现了负数，这是有问题的，要进一步探明原因

找出`Quantity`小于0的观察值

In [12]:
original_data[original_data["Quantity"]<0]

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
141,C536379,D,Discount,-1,12/1/2010 9:41,27.50,14527.0,United Kingdom
154,C536383,35004C,SET OF 3 COLOURED FLYING DUCKS,-1,12/1/2010 9:49,4.65,15311.0,United Kingdom
235,C536391,22556,PLASTERS IN TIN CIRCUS PARADE,-12,12/1/2010 10:24,1.65,17548.0,United Kingdom
236,C536391,21984,PACK OF 12 PINK PAISLEY TISSUES,-24,12/1/2010 10:24,0.29,17548.0,United Kingdom
237,C536391,21983,PACK OF 12 BLUE PAISLEY TISSUES,-24,12/1/2010 10:24,0.29,17548.0,United Kingdom
...,...,...,...,...,...,...,...,...
540449,C581490,23144,ZINC T-LIGHT HOLDER STARS SMALL,-11,12/9/2011 9:57,0.83,14397.0,United Kingdom
541541,C581499,M,Manual,-1,12/9/2011 10:28,224.69,15498.0,United Kingdom
541715,C581568,21258,VICTORIAN SEWING BOX LARGE,-5,12/9/2011 11:57,10.95,15311.0,United Kingdom
541716,C581569,84978,HANGING HEART JAR T-LIGHT HOLDER,-1,12/9/2011 11:58,1.25,17315.0,United Kingdom


观察发现这些数据的`InvoiceNo`都以C开头，这表示这些交易时无效的。要验证猜想，只需要看是否存在`InvoiceNo`不以C开头，且`Quantity`小于0的观察值

In [13]:
original_data[(original_data["Quantity"]<0)&(original_data["InvoiceNo"].str[0]!='C')]

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
2406,536589,21777,NaN,-10,12/1/2010 16:50,0.0,NaN,United Kingdom
4347,536764,84952C,NaN,-38,12/2/2010 14:42,0.0,NaN,United Kingdom
7188,536996,22712,NaN,-20,12/3/2010 15:30,0.0,NaN,United Kingdom
7189,536997,22028,NaN,-20,12/3/2010 15:30,0.0,NaN,United Kingdom
7190,536998,85067,NaN,-6,12/3/2010 15:30,0.0,NaN,United Kingdom
...,...,...,...,...,...,...,...,...
535333,581210,23395,check,-26,12/7/2011 18:36,0.0,NaN,United Kingdom
535335,581212,22578,lost,-1050,12/7/2011 18:38,0.0,NaN,United Kingdom
535336,581213,22576,check,-30,12/7/2011 18:38,0.0,NaN,United Kingdom
536908,581226,23090,missing,-338,12/8/2011 9:56,0.0,NaN,United Kingdom


虽然存在这类数据，但这类数据的`UnitPrice`似乎都为0。如果这样，这些数据显然无效的。要验证猜想，在上面一步的基础上，再添加限制条件 
```python
(original_data["UnitPrice"]!=0)
```

In [14]:
original_data[(original_data["Quantity"]<0)&(original_data["InvoiceNo"].str[0]!='C')&(original_data["UnitPrice"]!=0)]

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country


结果为空，说明我们猜想正确。

找出`UnitPrice`小于0的观察值

In [15]:
original_data[original_data["UnitPrice"]<0]

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
299983,A563186,B,Adjust bad debt,1,8/12/2011 14:51,-11062.06,NaN,United Kingdom
299984,A563187,B,Adjust bad debt,1,8/12/2011 14:52,-11062.06,NaN,United Kingdom


这两个观察值的描述都是调整坏帐，说明这两个观察值也是无效的。

评估数据结论
- 删除`Description`为空的观察值
- United Kingdom UK U.K.-> 英国
- USA United States->美国
- 删除`UnitPrice` `Quantity`小于0的观察值
- InvoiceDateDtype=object，实际上发票日期应为datetime; CustomerIDDtype = float，实际上应为string(object)

## 3. 清理数据

把清理后的数据写入`cleaned_data`

In [16]:
cleaned_data = original_data.copy()
cleaned_data.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom


把`InVoiceDate`的数据类型变为`datetime`

In [17]:
cleaned_data["InvoiceDate"] = pd.to_datetime(cleaned_data["InvoiceDate"])

In [18]:
cleaned_data["InvoiceDate"] 

0        2010-12-01 08:26:00
1        2010-12-01 08:26:00
2        2010-12-01 08:26:00
3        2010-12-01 08:26:00
4        2010-12-01 08:26:00
                 ...        
541904   2011-12-09 12:50:00
541905   2011-12-09 12:50:00
541906   2011-12-09 12:50:00
541907   2011-12-09 12:50:00
541908   2011-12-09 12:50:00
Name: InvoiceDate, Length: 541909, dtype: datetime64[ns]

把`CustomerID`的数据类型变为`object`

In [19]:
cleaned_data["CustomerID"] = cleaned_data["CustomerID"].astype(str)
cleaned_data["CustomerID"]

0         17850.0
1         17850.0
2         17850.0
3         17850.0
4         17850.0
           ...   
541904    12680.0
541905    12680.0
541906    12680.0
541907    12680.0
541908    12680.0
Name: CustomerID, Length: 541909, dtype: object

删除`CustomerID`结尾的`.0`

In [20]:
cleaned_data["CustomerID"] = cleaned_data["CustomerID"].str.slice(0,-2)
cleaned_data["CustomerID"]

0         17850
1         17850
2         17850
3         17850
4         17850
          ...  
541904    12680
541905    12680
541906    12680
541907    12680
541908    12680
Name: CustomerID, Length: 541909, dtype: object

删除Description为空的观察值

In [21]:
cleaned_data.dropna(subset = ["Description"],inplace=True)

In [22]:
cleaned_data["Description"].isnull().sum()

0

清理成功

United Kingdom UK U.K.-> 英国  
USA United States->美国

In [30]:
cleaned_data["Country"].replace(["UK","U.K."],"United Kingdom",inplace=True)
cleaned_data["Country"].replace("USA","United States",inplace=True)

In [31]:
cleaned_data["Country"].value_counts()

United Kingdom          494024
Germany                   9495
France                    8557
EIRE                      8196
Spain                     2533
Netherlands               2371
Belgium                   2069
Switzerland               2002
Portugal                  1519
Australia                 1259
Norway                    1086
Italy                      803
Channel Islands            758
Finland                    695
Cyprus                     622
Sweden                     462
Unspecified                446
Austria                    401
Denmark                    389
Japan                      358
Poland                     341
Israel                     297
United States              291
China                      288
Singapore                  229
Iceland                    182
Canada                     151
Greece                     146
Malta                      127
United Arab Emirates        68
European Community          61
RSA                         58
Lebanon 

把`Quantity`小于0的观察值删除，并检查替换后`Quantity`小于0的个数

In [33]:
cleaned_data = cleaned_data[cleaned_data["Quantity"] >=0 ]
len(cleaned_data[cleaned_data["Quantity"] < 0])

0

把`UnitPrice`小于0的观察值删除，并检查替换后`UnitPrice`小于0的个数

In [35]:
cleaned_data = cleaned_data[cleaned_data["UnitPrice"] >=0 ]
len(cleaned_data[cleaned_data["UnitPrice"] < 0])

0

## 4. 保存清理后的数据

把清理后的数据保存在新的文件中，命名为`e_commerce_cleaned.csv`

In [36]:
cleaned_data.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850,United Kingdom


In [37]:
cleaned_data.to_csv("e_commerce_cleaned.csv",index = False)